This notebook is part of [miubrt](https://github.com/meteo-ubonn/miubrt).

Copyright (c) [miubrt developers](https://github.com/meteo-ubonn/miubrt/blob/main/CONTRIBUTORS.txt). 
Distributed under the MIT License. See [LICENSE.txt](https://github.com/meteo-ubonn/miubrt/blob/main/LICENSE.txt) for more info.

# Create NetCDF4 output - Explore BoXPol/JuXPol
<div class="alert alert-info">

**User Info** <br>
 Global Data - User Input 
 
 start_time, end_time, location and sweep
 
 location = "BoXPol" or "JuXPol"  (X-band radar)
   
</div>
 

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import datetime as dt
import numpy as np
import dask
import wradlib as wrl
import xarray as xr

In [ ]:
# for using local development version, uncomment next lines otherwise installed `miubrt` version will be used
miubrt_dir = "../../"
if sys.path[0] != miubrt_dir:
     sys.path.insert(0, "../../")

In [ ]:
import miubrt as mrt

# Global Data - USER INPUT

In [ ]:
start_time = dt.datetime(2015, 7, 5, 14, 0)
end_time   = dt.datetime(2015, 7, 5, 17, 30, 0)
location   = "BoXPol"
# User Input End #

if location == "BoXPol":
    sname  = "n_rhi_lacros"
    sweep  = 0
elif location == "JuXPol":
    sname  = "RHI_05"
    sweep = 0
output_path = ""

# Create Netcdf Dataset of wanted radar data

In [ ]:
fname_raw = mrt.io.create_netcdf_dataset(location, sname, start_time, end_time, sweep, 
                                         outpath=output_path, engine='h5netcdf', chunks=None)
fname_raw

# Load and Explore Dataset

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
from matplotlib import gridspec
import hvplot
import hvplot.xarray
import holoviews as hv

## Load

In [ ]:
swp = xr.open_dataset(fname_raw,chunks={"time": 12})

In [ ]:
swp  = swp.chunk()
rh   = swp['range']
th   = swp['elevation']  
#
zh    = swp['DBZH']
zdr   = swp['ZDR']
kdp   = swp['KDP']
rhohv   = swp['RHOHV']
phidp   = swp['PHIDP']
zdr_off = zdr + 1.03
zh_off  = zh + 3.0

## Diagnostic Plots

In [ ]:
colors = ('#00f8f8', '#01b8fb', '#0000fa', '#00ff00', '#00d000', '#009400', 
          '#ffff00', '#f0cc00', '#ff9e00', '#ff0000', '#e40000', '#cc0000', 
          '#ff00ff', '#a06cd5')

dbz_levels=  [ -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
zdr_levels = [-1.,-0.5,-0.4,-0.3,-0.2,-0.1,0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0, 1.2]
kdp_levels = [-1., -0.5, -0.1, 0.0, 0.05, 0.10, 0.20, 0.30, 0.40, 0.60, 0.80, 1., 2., 3.]
rho_levels = [0.2,0.4,0.5, 0.7, 0.8, 0.94, 0.95, 0.96, 0.97, 0.98, 0.985, 0.99, 0.995, 1.0] 
#0.7,0.8, 0.85, 0.9, 0.92
phi_levels = [0, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]
phi_levels = [100, 105, 110, 115, 120, 125, 130, 140, 150, 160, 170, 180, 190, 200]
phi_levels = [-100, -90, -80, -70, -60, -50, -40, -30, 25, -20, -15, -10, -5, 0]

In [ ]:
t=16
boxminmax = [0,50,0,15]

##
xmin = boxminmax[0]
xmax = boxminmax[1]
ymin = boxminmax[2]
ymax = boxminmax[3]

title_string = swp.time.values[t]
az_angle     = str(int(swp.azimuth.values[0,0]))
#
fig = plt.figure(figsize=(20,15),constrained_layout=True)
gs = gridspec.GridSpec(2, 2,fig)
#
ax0 = fig.add_subplot(gs[0,0])
p0  = wrl.vis.plot_rhi(zh_off[t],r=rh,th=th,
                          levels=dbz_levels,colors=colors,
                          add_colorbar=True, 
                          cmap=False,
                          cbar_kwargs={'ticks': dbz_levels[0:-1]},
                          rf=1e3,ax=ax0)
ax0.set_xlim(xmin,xmax)
ax0.set_ylim(ymin,ymax)
ylabel = ax0.set_xlabel('Ground Range [km]')
ylabel = ax0.set_ylabel('Height [km]')
plt.title(title_string)

#
ax1 = fig.add_subplot(gs[0,1:])
p1  = wrl.vis.plot_rhi(zdr_off[t],r=rh,th=th,
                          levels=zdr_levels,colors=colors,
                          add_colorbar=True, 
                          cmap=False,
                          cbar_kwargs={'ticks': zdr_levels[0:-1]},
                          rf=1e3,ax=ax1)
ax1.set_xlim(xmin,xmax)
ax1.set_ylim(ymin,ymax)
ylabel = ax1.set_xlabel('Ground Range [km]')
ylabel = ax1.set_ylabel('Height [km]')
plt.title(title_string)
#
ax2 = fig.add_subplot(gs[1:,0])
p2  = wrl.vis.plot_rhi(rhohv[t],r=rh,th=th,
                          levels=rho_levels,colors=colors,
                          add_colorbar=True, 
                          cmap=False,
                          cbar_kwargs={'ticks': rho_levels[0:-1]},
                          rf=1e3,ax=ax2)
ax2.set_xlim(xmin,xmax)
ax2.set_ylim(ymin,ymax)
ylabel = ax2.set_xlabel('Ground Range [km]')
ylabel = ax2.set_ylabel('Height [km]')
plt.title(title_string)
#
ax3 = fig.add_subplot(gs[1:,1:])
p3  = wrl.vis.plot_rhi(phidp[t],r=rh,th=th,
                          levels=phi_levels,colors=colors,
                          add_colorbar=True, 
                          cmap=False,
                          cbar_kwargs={'ticks': phi_levels[0:-1]},
                          rf=1e3,ax=ax3)
ax3.set_xlim(xmin,xmax)
ax3.set_ylim(ymin,ymax)
ylabel = ax3.set_xlabel('Ground Range [km]')
ylabel = ax3.set_ylabel('Height [km]')
plt.title(title_string)

figname = location + "_" + az_angle + "_" + str(title_string) + ".png"
plt.savefig(figname)